In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"

In [2]:
police_box_4sido=pd.read_csv('치안센터_4개시.csv')

In [3]:
others_one_person = pd.read_csv('33.4개시도_1인가구수.csv')
others_basic_recipient = pd.read_csv('34.4개시도_기초수급자현황.csv')
others_sex_age_pop = pd.read_csv('35.4개시도_성연령별_총인구분포.csv')
others_foreigner_pop = pd.read_csv('36.4개시도_인구데이터(외국인).csv')
others_suicide = pd.read_csv('37.4개시도_자살율.csv')
others_fire = pd.read_csv('38.4개시도_화재발생통계.csv')

### 행정동 경계 geometry 데이터

In [4]:
행정동경계 = gpd.read_file('TL_SCCO_SIG.shp',encoding = 'euc-kr')
행정동경계

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,42110,Chuncheon-si,춘천시,"POLYGON ((1007462.095 2008948.792, 1007512.150..."
1,42130,Wonju-si,원주시,"POLYGON ((1038519.196 1944888.174, 1038552.691..."
2,42150,Gangneung-si,강릉시,"MULTIPOLYGON (((1136545.803 1962251.161, 11365..."
3,42170,Donghae-si,동해시,"MULTIPOLYGON (((1140184.975 1956251.627, 11401..."
4,42190,Taebaek-si,태백시,"POLYGON ((1133987.168 1927444.971, 1133980.895..."
...,...,...,...,...
245,43745,Jeungpyeong-gun,증평군,"POLYGON ((1003329.529 1867824.774, 1003386.941..."
246,43750,Jincheon-gun,진천군,"POLYGON ((994785.370 1890378.710, 994834.270 1..."
247,43760,Goesan-gun,괴산군,"POLYGON ((1026770.639 1883281.549, 1026781.261..."
248,43770,Eumseong-gun,음성군,"POLYGON ((1011533.109 1906176.344, 1011531.998..."


In [5]:
행정동_서울 = 행정동경계.iloc[140:165]
행정동_3개시 = 행정동경계[(행정동경계['SIG_KOR_NM'].str.contains('진주'))|행정동경계['SIG_KOR_NM'].str.contains('창원')|행정동경계['SIG_KOR_NM'].str.contains('수원')]


In [6]:
행정동_4개시 =pd.concat([행정동_서울,행정동_3개시]).reset_index(drop = True)
행정동_4개시.head()

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1..."


In [7]:
# 지역 이름 매핑
area_list = ['서울']*25 + ['수원']*4 + ['창원']*5 + ['진주']
행정동_4개시['지역'] = area_list

# 구 이름 통일
for i in range(25,34):
    행정동_4개시['SIG_KOR_NM'] = 행정동_4개시['SIG_KOR_NM'].apply(lambda x: x.split(" ")[-1])

행정동_4개시=행정동_4개시[['지역','SIG_KOR_NM','geometry']].rename(columns = {'SIG_KOR_NM':'구'})

### 4개시도_1인가구수 데이터 매핑
- 성별, 년도, 아파트, 비아파트 관련 feature

In [8]:
others_one_person.head()

,sido,sgg_nm,sx,age,date,single_person_hh,house_total_num,detached_house,apartment,townhouse,multiplex_house,house_in_commercial,not_in_house
0,서울특별시,종로구,계,합계,2017,23638,19260,11234,2992,887,3279,868,4378
1,서울특별시,종로구,계,20세 미만,2017,507,459,379,11,11,38,20,48
2,서울특별시,종로구,계,20~24,2017,2992,2519,1773,158,44,340,204,473
3,서울특별시,종로구,계,25~29,2017,3627,2845,1677,398,77,552,141,782
4,서울특별시,종로구,계,30~34,2017,2552,1970,1024,357,64,449,76,582


In [9]:
# 구 이름 통일
for i in range(len(others_one_person)):
    if len(others_one_person.loc[i,'sgg_nm']) != 2 and len(others_one_person.loc[i,'sgg_nm']) != 3 and len(others_one_person.loc[i,'sgg_nm']) != 4:
        others_one_person.loc[i,'sgg_nm'] = others_one_person.loc[i,'sgg_nm'][3:]

In [10]:
lis = list(others_one_person[others_one_person['apartment']=='X']['apartment'].index)
for i in lis:
    others_one_person.loc[i,'apartment'] = 0

In [11]:
others_one_person['apartment'] = others_one_person['apartment'].astype(int)
others_one_person['1인가구_비아파트'] = others_one_person['single_person_hh'] - others_one_person['apartment']
others_one_person.rename(columns= {'sgg_nm':'구','sx':'성별','date':'년도','apartment':'1인가구_아파트'},inplace = True)
others_one_person = others_one_person[['구','성별','년도','1인가구_아파트','1인가구_비아파트']]

In [12]:
others_one_person_groupby = others_one_person.groupby(['구','성별','년도']).agg({'1인가구_아파트':'sum','1인가구_비아파트':'sum'}).reset_index()

In [14]:
# # 2017,2018,2019년 외국인 총인원 count
# for i in range(len(others_one_person_groupby)):
#     for j in range(len(행정동_4개시)):
#         if others_one_person_groupby.loc[i,'구'] == 행정동_4개시.loc[j,'구']:
#             if others_one_person_groupby.loc[i,'성별'] =='남자':
#                 if others_one_person.loc[i,'년도'] == 2017:
#                     행정동_4개시.loc[j,'2017_남자_아파트'] = others_one_person_groupby.loc[i,'1인가구_아파트']
#                     행정동_4개시.loc[j,'2017_남자_비아파트'] = others_one_person_groupby.loc[i,'1인가구_비아파트']
#                 elif others_one_person.loc[i,'년도'] == 2018:
#                     행정동_4개시.loc[j,'2018_남자_아파트'] = others_one_person_groupby.loc[i,'1인가구_아파트']
#                     행정동_4개시.loc[j,'2018_남자_비아파트'] = others_one_person_groupby.loc[i,'1인가구_비아파트']
#                 elif others_one_person.loc[i,'년도'] == 2019:
#                     행정동_4개시.loc[j,'2019_남자_아파트'] = others_one_person_groupby.loc[i,'1인가구_아파트']
#                     행정동_4개시.loc[j,'2019_남자_비아파트'] = others_one_person_groupby.loc[i,'1인가구_비아파트']
#             elif others_one_person_groupby.loc[i,'성별'] =='여자':
#                 if others_one_person.loc[i,'년도'] == 2017:
#                     행정동_4개시.loc[j,'2017_여자_아파트'] = others_one_person_groupby.loc[i,'1인가구_아파트']
#                     행정동_4개시.loc[j,'2017_여자_비아파트'] = others_one_person_groupby.loc[i,'1인가구_비아파트']
#                 elif others_one_person.loc[i,'년도'] == 2018:
#                     행정동_4개시.loc[j,'2018_여자_아파트'] = others_one_person_groupby.loc[i,'1인가구_아파트']
#                     행정동_4개시.loc[j,'2018_여자_비아파트'] = others_one_person_groupby.loc[i,'1인가구_비아파트']
#                 elif others_one_person.loc[i,'년도'] == 2019:
#                     행정동_4개시.loc[j,'2019_여자_아파트'] = others_one_person_groupby.loc[i,'1인가구_아파트']
#                     행정동_4개시.loc[j,'2019_여자_비아파트'] = others_one_person_groupby.loc[i,'1인가구_비아파트']

### 4개시도_기초수급자현황 데이터 매핑
- 구와 년도별 기초수급자의 평균 기초수급자의 수를 feature
- 창원시와 진주시, 수원시는 구별로 되어 있지 않아 따로 DF 만들었음

In [15]:
others_basic_recipient.head()

,date,sido,sgg_nm,age,num
0,201706,서울특별시,종로구,4,4
1,201706,서울특별시,종로구,45,26
2,201706,서울특별시,종로구,46,55
3,201706,서울특별시,종로구,47,37
4,201706,서울특별시,종로구,48,51


In [16]:
others_basic_recipient = others_basic_recipient[['date','sido','sgg_nm','num']]
for i in range(len(others_basic_recipient)):
    if str(others_basic_recipient['date'][i])[0:4] == '2017': others_basic_recipient['date'][i] = 2017
    elif  str(others_basic_recipient['date'][i])[0:4] == '2018': others_basic_recipient['date'][i] = 2018
    elif  str(others_basic_recipient['date'][i])[0:4] == '2019': others_basic_recipient['date'][i] = 2019
    elif  str(others_basic_recipient['date'][i])[0:4] == '2020': others_basic_recipient['date'][i] = 2020
    elif  str(others_basic_recipient['date'][i])[0:4] == '2021': others_basic_recipient['date'][i] = 2021


In [17]:
기초수급자_4개시_평균 = others_basic_recipient.groupby(['sgg_nm','date']).agg({'num':"mean"}).reset_index().rename(columns = {'num':'평균_기초수급자','date':'년도','sgg_nm':'시군구'})

### 38.4개시도_화재발생통계 데이터 매핑
- 구별로 연도당 화재발생건수 feature

In [18]:
others_fire.head()

,yr,sido,sgg_nm,acc_num,dead_num,injr_num,tot_vic_num,prop_dmg,chattel_dmg,assts_dmg,dmg_per_acc
0,2018,경기도,수원시 권선구,108,0,18,18,1034047,756064,1790111,16575
1,2018,경기도,수원시 영통구,89,1,2,3,302241,466660,768901,8639
2,2018,경기도,수원시 장안구,92,0,4,4,203169,200131,403300,4384
3,2018,경기도,수원시 팔달구,81,0,17,17,941048,1031151,1972199,24348
4,2019,경기도,수원시 권선구,124,0,11,11,331803,472602,804405,6487


In [19]:
others_fire_groupby = others_fire.groupby(['sgg_nm','yr']).agg({'acc_num':'sum'}).reset_index()

In [20]:
for i in range(len(others_fire_groupby)):
    for j in range(len(행정동_4개시)):
        if others_fire_groupby.loc[i,'sgg_nm'] == 행정동_4개시.loc[j,'구'] and others_fire_groupby.loc[i,'yr'] == 2018:
            행정동_4개시.loc[j,'2018_화재'] = others_fire_groupby.loc[i,'acc_num']
        elif others_fire_groupby.loc[i,'sgg_nm'] == 행정동_4개시.loc[j,'구'] and others_fire_groupby.loc[i,'yr'] == 2019:
            행정동_4개시.loc[j,'2019_화재'] = others_fire_groupby.loc[i,'acc_num']
        elif others_fire_groupby.loc[i,'sgg_nm'] == 행정동_4개시.loc[j,'구'] and others_fire_groupby.loc[i,'yr'] == 2020:
            행정동_4개시.loc[j,'2020_화재'] = others_fire_groupby.loc[i,'acc_num']

### 36.4개시도_인구데이터(외국인)
- 구별,년도별 외국인 인구 수 feature

In [21]:
others_foreigner_pop.head()

,sido,sgg,foreigner_all(2017),foreigner_M(2017),foreigner_F(2017),foreigner_all(2018),foreigner_M(2018),foreigner_F(2018),foreigner_all(2019),foreigner_M(2019),foreigner_F(2019)
0,서울특별시,종로구,10979,4913,6066,12056,5536,6520,13159,5818,7341
1,서울특별시,중구,10265,4804,5461,11988,5735,6253,12559,6025,6534
2,서울특별시,용산구,16005,8712,7293,19126,10879,8247,20503,11621,8882
3,서울특별시,성동구,10436,5042,5394,11307,5403,5904,11774,5610,6164
4,서울특별시,광진구,18399,8323,10076,19729,9032,10697,20925,9474,11451


In [22]:
# 2017,2018,2019년 외국인 총인원 count
for i in range(len(others_foreigner_pop)):
    for j in range(len(행정동_4개시)):
        if others_foreigner_pop.loc[i,'sgg'] == 행정동_4개시.loc[j,'구']:
            행정동_4개시.loc[j,'2017_외국인'] = others_foreigner_pop.loc[i,'foreigner_all(2017)']
            행정동_4개시.loc[j,'2018_외국인'] = others_foreigner_pop.loc[i,'foreigner_all(2018)']
            행정동_4개시.loc[j,'2019_외국인'] = others_foreigner_pop.loc[i,'foreigner_all(2019)']

### 35.4개시도_성연령별_총인구분포 데이터 매핑

In [31]:
others_sex_age_pop.head()

,sido,sgg_nm,age,date,popu_num,popu_male_num,popu_female_num,popu_sx_rate,local_num,local_male_num,local_female_num,local_sx_rate
0,서울특별시,종로구,합계,2017,157277,76670,80607,95.1,146298,71757,74541,96.3
1,서울특별시,종로구,0~4세,2017,4137,2099,2038,103.0,4011,2039,1972,103.4
2,서울특별시,종로구,0세,2017,694,336,358,93.9,675,327,348,94.0
3,서울특별시,종로구,1세,2017,836,429,407,105.4,805,412,393,104.8
4,서울특별시,종로구,2세,2017,878,459,419,109.5,850,447,403,110.9


In [39]:
others_sex_age_pop['age'].unique()

array(['합계', '0~4세', '0세', '1세', '2세', '3세', '4세', '5~9세', '5세', '6세',
       '7세', '8세', '9세', '10~14세', '10세', '11세', '12세', '13세', '14세',
       '15~19세', '15세', '16세', '17세', '18세', '19세', '20~24세', '20세',
       '21세', '22세', '23세', '24세', '25~29세', '25세', '26세', '27세', '28세',
       '29세', '30~34세', '30세', '31세', '32세', '33세', '34세', '35~39세',
       '35세', '36세', '37세', '38세', '39세', '40~44세', '40세', '41세', '42세',
       '43세', '44세', '45~49세', '45세', '46세', '47세', '48세', '49세',
       '50~54세', '50세', '51세', '52세', '53세', '54세', '55~59세', '55세',
       '56세', '57세', '58세', '59세', '60~64세', '60세', '61세', '62세', '63세',
       '64세', '65~69세', '65세', '66세', '67세', '68세', '69세', '70~74세',
       '70세', '71세', '72세', '73세', '74세', '75~79세', '75세', '76세', '77세',
       '78세', '79세', '80~84세', '80세', '81세', '82세', '83세', '84세', '85세이상',
       '85~89세', '85세', '86세', '87세', '88세', '89세', '90~94세', '90세',
       '91세', '92세', '93세', '94세', '95~99세', '95세', '96세', '97세', '98세',
       '

### 37.4개시도_자살율 데이터 매핑
- 연령표준화 사망률: 인구구조가 다른 집단간의 사망 수준을 비교하기 위하여 연령구조가 사망률에 미치는 영향을 제거한 사망률
- 구별로 성별과 연령표준화를 feature

In [23]:
others_suicide.head()

,sido,sgg_nm,sx,dead_num_2017,dptt_2017,age_stndrd_dptt_2017,dead_num_2018,dptt_2018,age_stndrd_dptt_2018,dead_num_2019,dptt_2019,age_stndrd_dptt_2019
0,서울특별시,종로구,남자,33,45.3,37.2,22,30.3,28.7,30,41.9,32.0
1,서울특별시,종로구,여자,12,15.9,12.4,8,10.6,9.6,9,12.0,11.4
2,서울특별시,중구,남자,20,33.4,26.0,17,28.4,20.0,20,33.4,28.5
3,서울특별시,중구,여자,7,11.5,6.4,7,11.4,9.7,5,8.1,8.8
4,서울특별시,용산구,남자,36,33.7,29.8,33,31.0,26.8,27,25.4,19.6


In [24]:
others_suicide=others_suicide[['sgg_nm','sx','age_stndrd_dptt_2017','age_stndrd_dptt_2018','age_stndrd_dptt_2019']]

In [25]:
# 2017,2018,2019년 외국인 총인원 count
for i in range(len(others_suicide)):
    for j in range(len(행정동_4개시)):
        if others_suicide.loc[i,'sgg_nm'] == 행정동_4개시.loc[j,'구']:
            if others_suicide.loc[i,'sx'] =='남자':
                행정동_4개시.loc[j,'2017_남자_자살률'] = others_suicide.loc[i,'age_stndrd_dptt_2017']
                행정동_4개시.loc[j,'2018_남자_자살률'] = others_suicide.loc[i,'age_stndrd_dptt_2018']
                행정동_4개시.loc[j,'2019_남자_자살률'] = others_suicide.loc[i,'age_stndrd_dptt_2019']
            elif others_suicide.loc[i,'sx'] == '여자':
                행정동_4개시.loc[j,'2017_여자_자살률'] = others_suicide.loc[i,'age_stndrd_dptt_2017']
                행정동_4개시.loc[j,'2018_여자_자살률'] = others_suicide.loc[i,'age_stndrd_dptt_2018']
                행정동_4개시.loc[j,'2019_여자_자살률'] = others_suicide.loc[i,'age_stndrd_dptt_2019']                    

In [26]:
행정동_4개시.head()

,지역,구,geometry,2018_화재,2019_화재,2020_화재,2017_외국인,2018_외국인,2019_외국인,2017_남자_자살률,2018_남자_자살률,2019_남자_자살률,2017_여자_자살률,2018_여자_자살률,2019_여자_자살률
0,서울,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1...",254.0,232.0,217.0,10979.0,12056.0,13159.0,37.2,28.7,32.0,12.4,9.6,11.4
1,서울,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1...",275.0,213.0,185.0,10265.0,11988.0,12559.0,26.0,20.0,28.5,6.4,9.7,8.8
2,서울,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1...",211.0,238.0,192.0,16005.0,19126.0,20503.0,29.8,26.8,19.6,12.8,14.6,11.1
3,서울,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1...",212.0,194.0,187.0,10436.0,11307.0,11774.0,28.4,22.7,21.6,8.5,12.2,10.2
4,서울,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1...",203.0,195.0,186.0,18399.0,19729.0,20925.0,28.0,28.2,21.5,9.5,11.5,9.9
